In [1]:
import numpy as np

In [ ]:
def optimise_single_multi_mcc(y_pred, y_true, max_num_iterations=10, linspace_size=10, start=0.2, end=0.5, ):
    best_score = 0
    i = 0
    flag = True
    while max_num_iterations > i and flag:
        flag = False
        print("start and finish:", start, end)
        for t in np.linspace(start, end, linspace_size):
            ts, tm, fs, fm = confusion_matrix_B(y_pred, y_true, t)
            score = mcc_single_multi(ts, tm, fs, fm)
            if score > best_score:
                best_t = t
                best_score = score
                flag = True
        start = best_t * 0.75
        end = best_t * 1.25
        i += 1
    print("Best Score:", best_score)
    print("Best t:", best_t)
    return best_score, best_t

In [2]:
def mcc(tp, tn, fp, fn):
    mcc_num = (tp * tn) - (fp * fn)
    mcc_den = ((tp + fp) * (tp + fn) * (tn + fp) * (tn + fn))**0.5
    mcc = mcc_num / mcc_den if mcc_den else 0
    return mcc

def mcc_single_multi(ts, tm, fs, fm):
    # # single
    # pre_single = ts / (ts + fs) if (ts + fs) != 0 else 0
    # rec_single = ts / (ts + fm) if (ts + fm) != 0 else 0
    
    # # multi
    # pre_multi = tm / (tm + fm) if (tm + fm) != 0 else 0
    # rec_multi = tm / (tm + fs) if (tm + fs) != 0 else 0

    # acc = (tm + ts) / (tm + ts + fm + fs)

    # mcc
    mcc_num = (tm * ts) - (fm * fs)
    mcc_den = ((tm + fm) * (tm + fs) * (fm + ts) * (ts + fs))**0.5
    mcc = mcc_num / mcc_den if mcc_den != 0 else 0
    return mcc

In [3]:
def median_seq(seq):
    # parse a sequence (which will be the output from the model)
    # for every series of consecutive ones, leave only the median as one - the rest turn to 0
    # seq = [1,1,0,0,0,0,0,1,1,1,0,0,0,0,0,1,1,1,0,1,1]
    new_seq = np.zeros(len(seq))
    consecutive_ones_indices = []
    medians = []

    # marks if the first consecutive zeros that are less than the typical size is the first (meaning is the beginning of the first domain)
    for i in range(len(seq)):
        if seq[i] == 0:
            if len(consecutive_ones_indices) > 0:
                medians.append(int(np.median(consecutive_ones_indices)))
            consecutive_ones_indices = []

        if seq[i] == 1:
            consecutive_ones_indices.append(i)

    if len(consecutive_ones_indices) > 0:
        medians.append(int(np.median(consecutive_ones_indices)))

    for elt in medians:
        new_seq[elt] = 1

    return new_seq

In [4]:
def confusion_matrix_A(y_pred, y_true, t, margin=20):
    y_pred = np.where(y_pred > t, 1, 0)
    y_pred = median_seq(y_pred)
    tp, tn, fp, fn = 0, 0, 0, 0

    y_true_cp = np.copy(y_true)

    for i in range(len(y_pred)):
        window = y_true_cp[max(0, i-margin):min(len(y_true_cp), i+margin+1)]
        indices_window = list(range(max(0, i-margin), min(len(y_true_cp), i+margin+1)))
        if y_pred[i] == 1.0:
            if 1.0 in window:
                pos = np.where(window == 1.0)[0][0]
                j = indices_window[pos]
                y_true_cp[j] = 0.0
                tp += 1
            else:
                fp += 1

    for i in range(len(y_pred)):
        if y_pred[i] == 0.0:
            if  y_true_cp[i] == 1.0:
                fn += 1
            else:
                tn += 1

    return (tp, tn, fp, fn)

def confusion_matrix_B(y_pred, y_true, t):
    ts, tm, fs, fm = 0, 0, 0, 0
    for (pred, true) in zip(y_pred, y_true):
        pred = np.where(pred > t, 1, 0)
        pred = median_seq(pred)
        true_num = np.sum(true)
        pred_num = np.sum(pred)
        if int(true_num) == 0 and int(pred_num) == 0:
            ts += 1
        
        if int(true_num) > 0 and int(pred_num) > 0:
            tm += 1

        if int(true_num) == 0 and int(pred_num) > 0:
            fm += 1

        if int(true_num) > 0 and int(pred_num) == 0:
            fs += 1
    return (ts, tm, fs, fm)


In [15]:
y_pred = np.load('../data/cutoff_optimisation/all_y_pred.npy')
y_true = np.load('../data/cutoff_optimisation/all_y_true.npy')

In [6]:
# t = 0.5
# # for i in range(num_iterations):

# num_iterations = 2
# best_t = None
# best_score = 0

# start, end = 0.2, 0.6
# for i in range(num_iterations):
# 	print("start and finish:", start, end)
# 	for t in np.linspace(start, end, 3):
# 		tp, tn, fp, fn = confusion_matrix(y_pred, y_true, t, 20)
# 		score = mcc(tp, tn, fp, fn)
# 		if score > best_score:
# 			best_t = t
# 			best_score = score
# 	start = best_t * 0.75
# 	end = best_t * 1.25

In [9]:
# print("best_t", t)
# print("best_score", score)

In [8]:
tp, tn, fp, fn = confusion_matrix_A(y_pred, y_true, 0.35, 20)
score = mcc(tp, tn, fp, fn)
score

0.3876531710085079

# 2d

In [16]:
y_pred_2d = np.load('../data/cutoff_optimisation/all_y_pred_2d.npy')
y_true_2d = np.load('../data/cutoff_optimisation/all_y_true_2d.npy')

In [18]:
y_pred = y_pred_2d[0]
y_true = y_true_2d[0]

In [47]:
n = 678

tp, tn, fp, fn = confusion_matrix_A(y_pred[:n], y_true[:n], 0.2)
print(tp, tn, fp, fn)
print(mcc(tp, tn, fp, fn))


n = 1800
tp, tn, fp, fn = confusion_matrix_A(y_pred[:n], y_true[:n], 0.2)
print(tp, tn, fp, fn)
print(mcc(tp, tn, fp, fn))

d = ((-0.8158924398306318 + 0.8162045055898428) / (0.8158924398306318)) * 100
print(f"When including padding there is a {d:.2f}% increase in MCC")

2 675 1 0
0.8158924398306318
2 1797 1 0
0.8162694924569376
When including padding there is a 0.04% increase in MCC


In [43]:
def optimise_mcc(y_pred, y_true, margin=20, num_iterations=2, start=0.2, end=0.6):
	best_score, score = 0, 0
	for i in range(num_iterations):
		print("start and finish:", start, end)
		for t in np.linspace(start, end, 3):
			for (pred, true) in zip(y_pred, y_true):
				tp, tn, fp, fn = confusion_matrix_A(pred, true, t, margin)
				score += mcc(tp, tn, fp, fn)
			score = score / y_pred.shape[0]
			if score > best_score:
				best_t = t
				best_score = score
		start = best_t * 0.75
		end = best_t * 1.25
	print("Best Score:", best_score)
	print("Best t:", best_t)
	return best_score, best_t

def optimise_single_multi_mcc(y_pred, true, num_iterations=2, linspace_size=3, start=0.2, end=0.6, ):
	best_score = 0
	for i in range(num_iterations):
		print("start and finish:", start, end)
		for t in np.linspace(start, end, linspace_size):
			ts, tm, fs, fm = confusion_matrix_B(y_pred, true, t)
			score = mcc_single_multi(ts, tm, fs, fm)
			if score > best_score:
				best_t = t
				best_score = score
		start = best_t * 0.75
		end = best_t * 1.25
	print("Best Score:", best_score)
	print("Best t:", best_t)
	return best_score, best_t

In [44]:
optimise_single_multi_mcc(y_pred_2d, y_true_2d, 2, 3)

start and finish: 0.2 0.6
start and finish: 0.15000000000000002 0.25
Best Score: 0.5632578567451626
Best t: 0.2


(0.5632578567451626, 0.2)

In [46]:
optimise_single_multi_mcc(y_pred_2d, y_true_2d, 2, 10)

start and finish: 0.2 0.6
start and finish: 0.15000000000000002 0.25
Best Score: 0.5632578567451626
Best t: 0.2


(0.5632578567451626, 0.2)

In [30]:
optimise_mcc(y_pred_2d, y_true_2d)

start and finish: 0.2 0.6
start and finish: 0.15000000000000002 0.25
Best Score: 0.26047197450439896
Best t: 0.15000000000000002


(0.26047197450439896, 0.15000000000000002)

In [31]:
# reproduce mcc in colab

score = 0 
for (pred, true) in zip(y_pred_2d, y_true_2d):
	tp, tn, fp, fn = confusion_matrix_A(pred, true, 0.35, 20)
	score += mcc(tp, tn, fp, fn)
score = score / y_pred_2d.shape[0]
score

0.15281694945520788